In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\Male\GeneCpG\Hopkins
data = pd.read_csv('Hopkins_with_MethylomeDetails_Male.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\Male\GeneCpG\Hopkins


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,Category,cg02440279,cg02131967,...,cg08990112,cg24987754,cg01108057,cg11424763,cg08584264,cg02750308,cg00382771,cg13510423,cg22194345,cg13669748
0,200973410167_R05C01,3010,1,1,2,47.0,2,0,0.012232,0.015209,...,0.038912,0.856477,0.036409,0.907263,0.10837,0.546931,0.845835,0.307199,0.008555,0.060829


In [4]:
X = data.drop(['Sentrix','PATNO','HYS','Age','AgeCate','Category','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [5]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [6]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6363636363636364

In [7]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=2)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100


BorutaPy finished running.

Iteration: 	38 / 100
Confirmed: 	2
Tentative: 	0
Rejected: 	4342


BorutaPy(estimator=RandomForestClassifier(n_estimators=20,
                                          random_state=RandomState(MT19937) at 0x22139329E40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x22139329E40, verbose=1)

In [8]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  2


In [9]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [12]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
2098,cg03616148,1
4063,cg00314622,1
2628,cg04922955,2


In [13]:
Top50.to_csv ('Hopkins_BorutaMale_Top50.csv', index = False)
selected_rf_features.to_csv ('Hopkins_BorutaMale_All.csv', index = False)

#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
